## base64模块
base64的原理：
1. 准备一个包含64个字符的数组：
 - ['A', 'B', 'C', ... 'a', 'b', 'c', ... '0', '1', ... '+', '/']
2. 对二进制数据进行处理，每三个字节一组，一共3x8=24bit，每组划分为4小组，每小组正好6bit。
这样就得到4个数字作为索引，然后查表，获得相应的4个字符，就是编码后的字符串。
3. 如果要编码的二进制数据不是3的倍数，最后会剩下1个或2个字节，base64用\x00字节在末尾补足，
再在编码的末尾加上1个或2个=号，表示补了多少个字节，解码的时候回自动去掉。
4. 由于=字符也可能出现在base64编码中，但=用在URL、Cookie里面会造成歧义，所以，很多base64
会把=去掉，base64编码的长度永远是4的倍数，解码时加上=使base64字符串的长度变为4的倍数。

In [22]:
import base64

print(base64.b64encode(b'binary\x00string'))
print(base64.b64decode(b'YmluYXJ5AHN0cmluZw=='))
print(base64.b64encode(b'i\xb7\x1d\xfb\xef\xff'))
print(base64.urlsafe_b64encode(b'i\xb7\x1d\xfb\xef\xff'))
print(base64.urlsafe_b64decode(b'abcd--__'))

b'YmluYXJ5AHN0cmluZw=='
b'binary\x00string'
b'abcd++//'
b'abcd--__'
b'i\xb7\x1d\xfb\xef\xff'


In [9]:
ord('f')

102

In [12]:
print(0b1100110)

102


### 练习
请写一个能处理去掉=的base64解码函数：

In [178]:
import base64, re

def safe_base64_decode1(s):
    l = [ chr(ord('A')+ i) for i in range(26)] +\
        [chr(ord('a')+ i) for i in range(26)] +\
        [chr(ord('0')+ i) for i in range(10)] + ['+','/']
    s1 = ''
    n = 0
    for i in s:
        if chr(i) == '=':
            n = n + 1
        else:
            t = l.index(chr(i))
            t = str(bin(t))[2:].zfill(6)
            s1 = s1 + t
    rl = re.findall(r'[0-1]{8}', s1)
    return ''.join([chr(int(i,2)) for i in rl]).encode()

def safe_base64_decode(s):
    num = len(s)%4
    s += b'='*(4-num)
    return base64.b64decode(s)

In [181]:
# 测试:
assert b'abcd' == safe_base64_decode(b'YWJjZA==')
assert b'abcd' == safe_base64_decode(b'YWJjZA')
print('ok')

ok
